In [1]:
import itertools
import pathlib
import warnings

import bitstring
import gym
import gym_multiplexer  # noqa: F401
from lcs import Perception
from myst_nb import glue
from tabulate import tabulate
from IPython.display import HTML

from src.bayes_estimation import bayes_estimate
from src.bayes_plotting import plot_bayes_comparison
from src.decorators import repeat, get_from_cache_or_run
from src.discretized_experiments import *
from src.utils import build_plots_dir_path, build_cache_dir_path

plt.ioff()  # turn off interactive plotting
plt.style.use('../../../../src/phd.mplstyle')

root_dir = pathlib.Path().cwd().parent.parent.parent.parent
cwd_dir = pathlib.Path().cwd()

plot_dir = build_plots_dir_path(root_dir) / cwd_dir.parent.name / cwd_dir.name
cache_dir = build_cache_dir_path(root_dir) / cwd_dir.parent.name / cwd_dir.name

(section-internalizing-experiment-3)=
# Experiment 3 - Single step

To convert real numbers into integers, a $k$ bins discretizer is used. Its value is used to control the accuracy of generated rules. Thus, the input space of the environment can be calculated as $2k^n$, where k refers to the number of bins and $n$ the length of the MPX signal.


Example of discretizing real-valued input using 6 bins. A random perception signal of 6-bit rMPX extended with 1 bit (last column) was used. The size of the input space is $2\cdot6^6=93312$ unique states.

In [2]:
RMPX_BINS = 10
RMPX_SIZE = 3
CTRL_BITS = 1

rmpx = gym.make(f'real-multiplexer-{RMPX_SIZE}bit-v0')

_range, _low = (rmpx.observation_space.high - rmpx.observation_space.low, rmpx.observation_space.low)

RMPX_STEP = _range / RMPX_BINS


class RealMultiplexerUtils:
    def __init__(self, size, ctrl_bits, bins, _range, _threshold=0.5):
        self._size = size
        self._ctrl_bits = ctrl_bits
        self._bins = bins
        self._step = _range / bins
        self._threshold = _threshold

        self._attribute_values = [list(range(0, bins))] * (size) + [[0, bins]]
        self._input_space = itertools.product(*self._attribute_values)
        self.state_mapping = {idx: s for idx, s in enumerate(self._input_space)}
        self.state_mapping_inv = {v: k for k, v in self.state_mapping.items()}

    def discretize(self, obs, _type=int):
        r = (obs + np.abs(_low)) / _range
        b = (r * RMPX_BINS).astype(int)
        return b.astype(_type).tolist()

    def reverse_discretize(self, discretized):
        return discretized * self._step[:len(discretized)]

    def get_transitions(self):
        transitions = []

        initial_dstates = [list(range(0, self._bins))] * (self._size)
        for d_state in itertools.product(*initial_dstates):
            correct_answer = self._get_correct_answer(d_state)

            if correct_answer == 0:
                transitions.append((d_state + (0,), 0, d_state + (self._bins,)))
                transitions.append((d_state + (0,), 1, d_state + (0,)))
            else:
                transitions.append((d_state + (0,), 0, d_state + (0,)))
                transitions.append((d_state + (0,), 1, d_state + (self._bins,)))

        return transitions

    def _get_correct_answer(self, discretized):
        estimated_obs = self.reverse_discretize(discretized)
        bits = bitstring.BitArray(estimated_obs > self._threshold)
        _ctrl_bits = bits[:self._ctrl_bits]
        _data_bits = bits[self._ctrl_bits:]

        return int(_data_bits[_ctrl_bits.uint])


rmpx_utils = RealMultiplexerUtils(RMPX_SIZE, CTRL_BITS, RMPX_BINS, _range)
# print(f"Possible states: {len(rmpx_utils.state_mapping)}")

In [3]:
# metrics
def generalization_score(pop):
    # Compute proportion of wildcards in classifier condition across all classifiers
    wildcards = sum(1 for cl in pop for cond in cl.condition if
                    cond == '#' or (
                            hasattr(cond, 'symbol') and cond.symbol == '#'))
    all_symbols = sum(len(cl.condition) for cl in pop)
    return wildcards / all_symbols


def rmpx_knowledge(population, env):
    reliable = [c for c in population if c.is_reliable()]
    nr_correct = 0

    for start, action, end in rmpx_utils.get_transitions():
        p0 = Perception([str(el) for el in start])
        p1 = Perception([str(el) for el in end])

        if any([True for cl in reliable if cl.predicts_successfully(p0, action, p1)]):
            nr_correct += 1

    return nr_correct / len(rmpx_utils.get_transitions())


def rmpx_metrics_collector(agent, env):
    population = agent.population
    return {
        'pop': len(population),
        'knowledge': rmpx_knowledge(population, env),
        'generalization': generalization_score(population)
    }


# DynaQ helpers
def rmpx_perception_to_int(p0, discretize=True):
    if discretize:
        p0 = rmpx_utils.discretize(p0)

    return rmpx_utils.state_mapping_inv[tuple(p0)]


def dynaq_rmpx_knowledge_calculator(model, env):
    all_transitions = 0
    nr_correct = 0

    for p0, a, p1 in rmpx_utils.get_transitions():
        s0 = rmpx_perception_to_int(p0, discretize=False)
        s1 = rmpx_perception_to_int(p1, discretize=False)

        all_transitions += 1
        if s0 in model and a in model[s0] and model[s0][a][0] == s1:
            nr_correct += 1

    return nr_correct / len(rmpx_utils.get_transitions())

In [4]:
class DiscretizedWrapper(gym.ObservationWrapper):
    def observation(self, obs):
        return rmpx_utils.discretize(obs, _type=str)


class SingleStateWrapper(DiscretizedWrapper):
    def observation(self, obs):
        return rmpx_utils.state_mapping_inv[tuple(map(int, obs))]


common_params = {
    'classifier_length': RMPX_SIZE + 1,
    'possible_actions': 2,
    'learning_rate': 0.1,
    'metrics_trial_freq': 100,
    'metrics_fcn': rmpx_metrics_collector,
    'trials': 15_000
}

yacs_params = {
    'trace_length': 3,
    'estimate_expected_improvements': False,
    'feature_possible_values': [{str(i) for i in range(RMPX_BINS)}] * RMPX_SIZE + [{'0', '10'}]
}

dynaq_params = {
    'q_init': np.zeros((len(rmpx_utils.state_mapping), 2)),
    'model_init': {},
    'knowledge_fcn': dynaq_rmpx_knowledge_calculator,
    'epsilon': 0.5
}

NUM_EXPERIMENTS = 50
USE_RAY = True


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_{RMPX_SIZE}bit/acs.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_rmpx_with_acs():
    return single_acs_experiment(
        env_provider=lambda: DiscretizedWrapper(rmpx),
        trials=common_params['trials'],
        classifier_length=common_params['classifier_length'],
        possible_actions=common_params['possible_actions'],
        learning_rate=common_params['learning_rate'],
        metrics_trial_freq=common_params['metrics_trial_freq'],
        metrics_fcn=common_params['metrics_fcn'])


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_{RMPX_SIZE}bit/acs2.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_rmpx_with_acs2():
    return single_acs2_experiment(
        env_provider=lambda: DiscretizedWrapper(rmpx),
        trials=common_params['trials'],
        classifier_length=common_params['classifier_length'],
        possible_actions=common_params['possible_actions'],
        learning_rate=common_params['learning_rate'],
        do_ga=False,
        initial_q=0.5,
        metrics_trial_freq=common_params['metrics_trial_freq'],
        metrics_fcn=common_params['metrics_fcn']
    )


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_{RMPX_SIZE}bit/acs2_ga.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_rmpx_with_acs2_ga():
    return single_acs2_experiment(
        env_provider=lambda: DiscretizedWrapper(rmpx),
        trials=common_params['trials'],
        classifier_length=common_params['classifier_length'],
        possible_actions=common_params['possible_actions'],
        learning_rate=common_params['learning_rate'],
        do_ga=True,
        initial_q=0.5,
        metrics_trial_freq=common_params['metrics_trial_freq'],
        metrics_fcn=common_params['metrics_fcn']
    )


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_{RMPX_SIZE}bit/yacs.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_rmpx_with_yacs():
    return single_yacs_experiment(
        env_provider=lambda: DiscretizedWrapper(rmpx),
        trials=common_params['trials'],
        classifier_length=common_params['classifier_length'],
        possible_actions=common_params['possible_actions'],
        learning_rate=common_params['learning_rate'],
        trace_length=yacs_params['trace_length'],
        estimate_expected_improvements=yacs_params['estimate_expected_improvements'],
        feature_possible_values=yacs_params['feature_possible_values'],
        metrics_trial_freq=common_params['metrics_trial_freq'],
        metrics_fcn=common_params['metrics_fcn']
    )


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_{RMPX_SIZE}bit/dynaq.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_rmpx_with_dynaq():
    return single_dynaq_experiment(
        env_provider=lambda: SingleStateWrapper(DiscretizedWrapper(rmpx)),
        trials=common_params['trials'],
        q_init=dynaq_params['q_init'],
        model_init=dynaq_params['model_init'],
        epsilon=dynaq_params['epsilon'],
        learning_rate=common_params['learning_rate'],
        knowledge_fcn=dynaq_params['knowledge_fcn'],
        metrics_trial_freq=common_params['metrics_trial_freq']
    )


@get_from_cache_or_run(cache_path=f'{cache_dir}/corridor/dynaq.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_corridor_with_dynaq():
    return single_dynaq_experiment(
        env_provider=lambda: SingleStateWrapper(CorridorObservationWrapper(corridor)),
        trials=common_params['trials'],
        q_init=dynaq_params['q_init'],
        model_init=dynaq_params['model_init'],
        epsilon=dynaq_params['epsilon'],
        learning_rate=common_params['learning_rate'],
        knowledge_fcn=dynaq_params['knowledge_fcn'],
        metrics_trial_freq=common_params['metrics_trial_freq']
    )


# Run computations
rmpx_acs_runs = run_rmpx_with_acs()
rmpx_acs2_runs = run_rmpx_with_acs2()
rmpx_acs2_ga_runs = run_rmpx_with_acs2_ga()
rmpx_yacs_runs = run_rmpx_with_yacs()
rmpx_dynaq_runs = run_rmpx_with_dynaq()

# Collect metrics to single dataframe
metrics_df = pd.concat([
    *[parse_lcs_metrics('acs', metrics) for _, metrics in rmpx_acs_runs],
    *[parse_lcs_metrics('acs2', metrics) for _, metrics in rmpx_acs2_runs],
    *[parse_lcs_metrics('acs2_ga', metrics) for _, metrics in rmpx_acs2_ga_runs],
    *[parse_lcs_metrics('yacs', metrics) for _, metrics in rmpx_yacs_runs],
    *[parse_dyna_metrics('dynaq', metrics) for _, _, metrics in rmpx_dynaq_runs],
])
metrics_df.set_index(['agent', 'trial'], inplace=True)

# Average them by agent and trial
metrics_averaged_df = metrics_df.groupby(['agent', 'trial']).mean()

## Plot

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    plot_comparison(metrics_averaged_df, plot_filename=f'{plot_dir}/rmpx_3bit_performance.png')

:::{figure-md} rmpx3bit-discretized-performance-fig
<img src="../../../_static/plots/3_internalizing/discretization/rmpx_3bit_performance.png">

Performance of 3bit discretized rMPX with 10 bins.
:::

## Statistical verification

```{admonition} Hypothesis testing
:class: tip
The same plot as in the latest IEEE submission
```

In [6]:
agents = ['acs', 'acs2', 'acs2_ga', 'yacs', 'dynaq']


def build_models(df: pd.DataFrame, field: str):
    results = {}

    for agent in agents:
        last_trial = df.reset_index(1).query(f'agent == "{agent}"')['trial'].max()
        data_arr = df.query(f'agent == "{agent}" and trial == {last_trial}')[field].to_numpy()
        bayes_model = bayes_estimate(data_arr)
        results[agent] = bayes_model['mu']

    return results


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_3bit/bayes/population.dill')
def build_population_model(df: pd.DataFrame):
    return build_models(df, 'population')


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_3bit/bayes/knowledge.dill')
def build_knowledge_models(df: pd.DataFrame):
    return build_models(df, 'knowledge')


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_3bit/bayes/generalization.dill')
def build_generalization_models(df: pd.DataFrame):
    return build_models(df, 'generalization')


@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx_3bit/bayes/timing.dill')
def build_timing_models(df: pd.DataFrame):
    return build_models(df, 'time')


population_models = build_population_model(metrics_df)
knowledge_models = build_knowledge_models(metrics_df)
generalization_models = build_generalization_models(metrics_df)
timing_models = build_timing_models(metrics_df)

# prepare table
print_row = lambda r: f'{round(r.mean(), 3)} ± {round(r.std(), 3)}'

payload_df = []
table_data = []

for agent in agents:
    # prepare data frame for visualization
    payload_df.append({
        'agent': agent,
        'knowledge': knowledge_models[agent].mean(),
        'generalization': generalization_models[agent].mean(),
        'population': population_models[agent].mean(),
        'time': timing_models[agent].mean()
    })

    # add data to table
    table_data.append([agent.upper(),
                       print_row(knowledge_models[agent]),
                       print_row(generalization_models[agent]),
                       print_row(population_models[agent]),
                       print_row(timing_models[agent])])

rmpx_bayes_df = pd.DataFrame(payload_df).set_index('agent')

bayes_table = tabulate(table_data,
                       headers=['', 'Knowledge', 'Generalization', 'Population', 'Trial time'],
                       tablefmt="html", stralign='right')

glue('ch33_1_bayes_table', HTML(bayes_table), display=False)
glue('ch33_1_bayes_fig', plot_bayes_comparison(rmpx_bayes_df, 'rMPX', agents, plot_filename=f'{plot_dir}/rmpx_3bit_bayes.png'), display=False)

,Knowledge,Generalization,Population,Trial time
acs,0.0 ± 0.0,0.875 ± 0.001,4.0 ± 0.001,0.001 ± 0.001
acs2,0.998 ± 0.001,0.174 ± 0.001,1622.405 ± 7.313,0.007 ± 0.001
acs2_ga,1.0 ± 0.001,0.318 ± 0.001,977.517 ± 5.966,0.006 ± 0.001
yacs,0.978 ± 0.001,0.106 ± 0.005,1160.178 ± 6.192,0.03 ± 0.001
dynaq,0.998 ± 0.001,0.0 ± 0.0,2000.0 ± 0.001,0.0 ± 0.0


````{tabbed} Bayes Table
{glue:}`ch33_1_bayes_table`
````

````{tabbed} Radar plot
```{glue:figure} ch33_1_bayes_fig
:name: "ch33_1_bayes_fig"
Radar plot
```
````

## Observations